# Langfuse 프롬프트 관리 

---

## 환경 설정 및 준비

### (1) Env 환경변수

- .env 파일
    ```markdown
    LANGFUSE_SECRET_KEY=sk-
    LANGFUSE_PUBLIC_KEY=pk-
    # 🇪🇺 EU region
    LANGFUSE_HOST=https://cloud.langfuse.com
    ```


In [ ]:
from dotenv import load_dotenv
load_dotenv()

### (2) 기본 라이브러리

In [ ]:
import os
from glob import glob
from pprint import pprint
import json
import warnings
warnings.filterwarnings("ignore")

### (3) Langfuse 콜백 핸들러 설정

In [ ]:
from langfuse.langchain import CallbackHandler 

# LangChain 콜백 핸들러 생성
langfuse_handler = CallbackHandler()

### (4) Langfuse 클라이언트 설정

In [ ]:
from langfuse import get_client

# Langfuse 클라이언트 초기화
langfuse = get_client()

# 연결 테스트
assert langfuse.auth_check()

---

## 프롬프트 관리 개요

Langfuse는 **프롬프트 CMS(Content Management System)** 기능을 제공

- **버전 관리**: 프롬프트의 모든 변경사항을 추적하고 롤백 가능
- **협업**: 팀원들과 함께 프롬프트를 편집하고 관리
- **배포 관리**: 라벨을 통해 코드 변경 없이 환경별 배포
- **성능 모니터링**: 프롬프트 버전별 성능 메트릭 비교
- **실시간 테스트**: 플레이그라운드에서 즉시 테스트 가능

---

## 1. 프롬프트 생성

### 1.1 텍스트 프롬프트 생성

In [ ]:
# 텍스트 프롬프트 생성
langfuse.create_prompt(
    name="movie-critic",  # 프롬프트 이름
    type="text",          
    prompt="{{criticLevel}} 영화 평론가로서, {{movie}}를 어떻게 생각하시나요?",
    labels=["production"],           # 프로덕션 레이블
    tags=["movie", "qa", "text"],    # 태그
    config={
        "model": "gpt-4.1-mini",
        "temperature": 0.7,
        "max_tokens": 500
    }
)

### 1.2 챗 프롬프트 생성

In [ ]:
# 챗 프롬프트 생성
langfuse.create_prompt(
    name="movie-critic-chat",  # 프롬프트 이름
    type="chat",          
    prompt=[
        {
            "role": "system",
            "content": "당신은 {{criticLevel}} 영화 평론가입니다."
        },
        {
            "role": "user",
            "content": "영화 {{movie}}를 어떻게 생각하시나요?"
        }
    ],
    labels=["production"],       # 프로덕션 레이블
    tags=["movie", "qa", "chat"],    # 태그
    config={
        "model": "gpt-4.1-mini",
        "temperature": 0.7,
        "max_tokens": 500
    }
)

### 1.3 메시지 플레이스홀더가 있는 챗 프롬프트

In [ ]:
# 메시지 플레이스홀더를 포함한 챗 프롬프트
langfuse.create_prompt(
    name="movie-critic-with-history",
    type="chat",
    prompt=[
        {
            "role": "system",
            "content": "당신은 {{criticLevel}} 영화 평론가입니다."
        },
        {
            "type": "placeholder",
            "name": "chat_history"  # 대화 히스토리 삽입 지점
        },
        {
            "role": "user",
            "content": "영화 {{movie}}에 대해 어떻게 생각하시나요?"
        }
    ],
    labels=["production"],
    tags=["movie", "qa", "chat", "history"]
)

### **[실습 1]**
텍스트 기반 프롬프트와 chat 기반 프롬프트를 각각 구현하고, Langfuse UI에서 확인하세요.

In [ ]:
# 텍스트 프롬프트 생성
# 여기에 코드를 작성하세요

<details>
<summary>💡 정답 보기</summary>

```python
# 간단한 문서 요약용 텍스트 프롬프트 생성
langfuse.create_prompt(
    name="document-summarizer",
    type="text",
    prompt="""다음 {{document_type}}를 {{summary_length}} 단어 이내로 요약해주세요:

제목: {{document_title}}

내용:
{{document_content}}

{{summary_style}} 형태로 요약해주세요.""",
    labels=["production"],
    tags=["document", "summary", "simple", "text"],
    config={
        "model": "gpt-4.1-mini",
        "temperature": 0.2,
        "max_tokens": 800
    }
)
```
</details>


In [ ]:
# 챗 프롬프트 생성
# 여기에 코드를 작성하세요

<details>
<summary>💡 정답 보기</summary>

```python
# 간단한 문서 요약용 챗 프롬프트 생성
langfuse.create_prompt(
    name="document-summarizer-chat",
    type="chat",
    prompt=[
        {
            "role": "system",
            "content": "당신은 문서 요약 전문가입니다. {{summary_style}} 형태로 {{summary_length}} 단어 이내로 요약해주세요."
        },
        {
            "role": "user",
            "content": """다음 {{document_type}}를 요약해주세요:

제목: {{document_title}}

내용:
{{document_content}}"""
        }
    ],
    labels=["production"],
    tags=["document", "summary", "chat", "simple"],
    config={
        "model": "gpt-4.1-mini",
        "temperature": 0.2,
        "max_tokens": 800
    }
)
```
</details>


---

## 2. 프롬프트 활용

### 2.1 기본 프롬프트 가져오기

In [ ]:
# 프로덕션 버전 가져오기
prompt = langfuse.get_prompt("movie-critic")

# 프롬프트 정보 출력
print(f"모델: {prompt.config['model']}")
print(f"온도: {prompt.config['temperature']}")
print(f"라벨: {prompt.labels}")
print(f"태그: {prompt.tags}")
print(f"프롬프트: {prompt.prompt}")
print("-" * 100)

# 랭체인 호환 프롬프트 출력
print(prompt.get_langchain_prompt())

### 2.2 compile 메서드 사용

- compile 메서드로 변수 삽입

In [ ]:
# compile 메서드로 변수 삽입
compiled_prompt = prompt.compile(criticLevel="전문가", movie="인셉션")
print(compiled_prompt)

### 2.3 챗 프롬프트 가져오기 및 컴파일

In [ ]:
# 챗 프롬프트 가져오기
chat_prompt = langfuse.get_prompt("movie-critic-chat", type="chat")

# 챗 프롬프트 정보 출력
print(f"모델: {chat_prompt.config['model']}")
print(f"온도: {chat_prompt.config['temperature']}")
print(f"라벨: {chat_prompt.labels}")
print(f"태그: {chat_prompt.tags}")
print(f"프롬프트: {chat_prompt.prompt}")
print("-" * 100)

# 랭체인 호환 프롬프트 출력
print(chat_prompt.get_langchain_prompt())

In [ ]:
# 챗 프롬프트 컴파일
compiled_chat_prompt = chat_prompt.compile(criticLevel="전문가", movie="인셉션")
print(compiled_chat_prompt)

### **[실습 2]**
"movie-critic-chat" 프롬프트를 Langfuse에서 가져와서 내용을 출력하고, compile 메서드를 사용해 변수에 적절한 값을 추가해보세요.

In [ ]:
# chat 프롬프트 가져오기 및 컴파일
# 여기에 코드를 작성하세요

<details>
<summary>💡 정답 보기</summary>

```python
# chat 프롬프트 가져오기 및 컴파일
chat_prompt = langfuse.get_prompt("movie-critic-chat", type="chat")

# 대화 히스토리 정의
chat_history = [
    {"role": "user", "content": "안녕하세요!"},
    {"role": "assistant", "content": "안녕하세요! 영화에 대해 이야기해볼까요?"}
]

# 변수와 플레이스홀더를 모두 컴파일
compiled_chat_prompt = chat_prompt.compile(
    criticLevel="전문가", 
    movie="인셉션"
)

for message in compiled_chat_prompt:
    print(message)
    print("-" * 20)
```
</details>


---

## 3. 프롬프트 버전 관리

### 3.1 새로운 버전 생성

In [ ]:
# 새로운 버전 생성 (같은 이름 사용)
langfuse.create_prompt(
    name="movie-critic",  # 같은 이름 사용
    type="text",          
    prompt="당신은 {{criticLevel}} 영화 평론가입니다.\n\n영화 {{movie}}에 대한 상세한 분석을 제공해주세요. 연출, 연기, 스토리, 시각적 효과를 포함하여 평가해주세요.",
    labels=["production"],       # 프로덕션 레이블
    tags=["movie", "qa", "text", "detailed"],    # 태그 업데이트
    config={
        "model": "gpt-4.1",  # 모델 업그레이드
        "temperature": 0.7,
        "max_tokens": 1000  # 토큰 수 증가
    }
)

### 3.2 특정 버전 가져오기

In [ ]:
# 특정 버전 가져오기
prompt_v1 = langfuse.get_prompt("movie-critic", version=1)
prompt_v2 = langfuse.get_prompt("movie-critic", version=2)

# 버전별 비교
print(f"V1 프롬프트: {prompt_v1.prompt}")
print(f"V2 프롬프트: {prompt_v2.prompt}")
print(f"V1 모델: {prompt_v1.config['model']}")
print(f"V2 모델: {prompt_v2.config['model']}")

### 3.3 라벨 관리

In [ ]:
# 특정 라벨로 프롬프트 생성 (같은 이름을 사용하면 새로운 버전으로 생성됨)
langfuse.create_prompt(
    name="movie-critic-chat",
    type="chat",
    prompt=[
        {
            "role": "system",
            "content": "당신은 {{criticLevel}} 영화 평론가입니다. 상세하고 전문적인 분석을 제공해주세요."
        },
        {
            "role": "user",
            "content": "영화 {{movie}}에 대한 평론을 작성해주세요."
        }
    ],
    labels=["staging"],  # staging 환경용
    tags=["movie", "qa", "chat", "detailed"]
)

# 라벨별 프롬프트 가져오기
prompt_production = langfuse.get_prompt("movie-critic-chat", label="production")
prompt_staging = langfuse.get_prompt("movie-critic-chat", label="staging")
prompt_latest = langfuse.get_prompt("movie-critic-chat", label="latest")

In [ ]:
# 라벨별 프롬프트 출력
print(f"Production 프롬프트: {prompt_production.prompt}")
print("-" * 100)
print(f"Staging 프롬프트: {prompt_staging.prompt}")
print("-" * 100)
print(f"Latest 프롬프트: {prompt_latest.prompt}")

### **[실습 3]**
"movie-critic-chat" 프롬프트를 수정하고, labels 속성은 "staging"으로 지정한 후, staging 버전을 가져와서 내용을 출력하세요.

In [ ]:
# staging 라벨 생성
# 여기에 코드를 작성하세요

# staging 라벨 가져오기
# 여기에 코드를 작성하세요

<details>
<summary>💡 정답 보기</summary>

```python
# staging 라벨 생성
langfuse.create_prompt(
    name="movie-critic-chat",
    type="chat",
    prompt=[
        {
            "role": "system",
            "content": "당신은 {{criticLevel}} 영화 평론가입니다. 일반적이고 상세하며 전문적인 분석을 제공해주세요."
        },
        {
            "role": "user",
            "content": "영화 {{movie}}에 대한 평론을 작성해주세요."
        }
    ],
    labels=["staging"],  # staging 환경용
    tags=["movie", "qa", "chat", "detailed"]
)

# staging 라벨 가져오기
prompt_staging = langfuse.get_prompt(
    name="movie-critic-chat",
    label="staging"
)

# 라벨별 프롬프트 출력
print(f"Staging 프롬프트: {prompt_staging.prompt}")
```
</details>


---

## 4. LangChain과의 통합

### 4.1 텍스트 프롬프트와 LangChain 통합

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Langfuse 프롬프트를 LangChain과 통합
prompt = langfuse.get_prompt("movie-critic", label="production")

langchain_prompt = PromptTemplate.from_template(
    prompt.get_langchain_prompt()
)
langchain_prompt.metadata = {"langfuse_prompt": prompt}   # Langfuse 자동 링크를 위한 메타데이터

# 모델 초기화 (프롬프트 설정 사용)
model = ChatOpenAI(
    model=prompt.config.get("model", "gpt-4.1-mini"),
    temperature=prompt.config.get("temperature", 0.7),
    max_completion_tokens=prompt.config.get("max_tokens", 500)
)

# 체인 생성 및 실행
chain = langchain_prompt | model
response = chain.invoke(
    input={"criticLevel": "전문가", "movie": "인셉션"},
    config={"callbacks": [langfuse_handler]}  # Langfuse 트레이싱을 위한 콜백
)

print(response.content)

### 4.2 챗 프롬프트와 LangChain 통합

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 챗 프롬프트 통합
chat_prompt = langfuse.get_prompt("movie-critic-chat", label="production", type="chat")

langchain_chat_prompt = ChatPromptTemplate.from_messages(
    chat_prompt.get_langchain_prompt()
)
langchain_chat_prompt.metadata = {"langfuse_prompt": chat_prompt}

# 체인 실행
chain = langchain_chat_prompt | model
response = chain.invoke(
    input={"criticLevel": "전문가", "movie": "인셉션"},
    config={"callbacks": [langfuse_handler]}
)

print(response.content)